In [ ]:
from typing import cast

from qonscious.adapters.aer_sampler_adapter import AerSamplerAdapter
from qonscious.adapters.backend_adapter import BackendAdapter
from qonscious.core import MeritComplianceCheck, ScorableFigureOfMeritResult
from qonscious.core.executor import run_conditionally
from qonscious.foms.packed_chsh import PackedCHSHTest

In [ ]:
def chsh_score_over(threshold: float):
    return lambda r: cast("ScorableFigureOfMeritResult", r)["score"] > threshold

check_chsh_is_ok = check = MeritComplianceCheck(
    figure_of_merit=PackedCHSHTest(),
    decision_function=chsh_score_over(2.8), # In the Aer simulator we can get up to 2.828
)

In [ ]:
# Neither the on_pass nor the on_fail callbacks run any circuits, they just print the result
# of the figure of merit checks

def on_pass(backendAdapter : BackendAdapter, figureOfMeritResults):
    firstFoMResult = cast("ScorableFigureOfMeritResult",figureOfMeritResults[0])
    print("CHSH test passed!")
    print(f"Score: {firstFoMResult['score']:.3f}")
    return None

def on_fail(backendAdapter : BackendAdapter, figureOfMeritResults):
    firstFoMResult = cast("ScorableFigureOfMeritResult",figureOfMeritResults[0])
    print("CHSH test failed!")
    print(f"Score: {firstFoMResult['score']:.3f}")
    return None

In [ ]:
backend = AerSamplerAdapter()

print("Running CHSH test with passing threshold...\n")
run_conditionally(
    backend_adapter=backend,
    checks= [check_chsh_is_ok],
    on_pass=on_pass,
    on_fail=on_fail,
    shots=2048
)